In [1]:
import numpy as np
import pandas as pd 

In [2]:
data_path = "gender_data/agg_df_1.csv"

df = pd.read_csv(data_path)

In [3]:
df.shape

(885, 90)

In [4]:
# display all columns in the dataframe

pd.set_option('display.max_columns', None)

In [5]:
df.head()

,userCode,age,company,gender,total_trips,avg_flight_price,avg_distance,avg_flight_time,avg_days,avg_hotel_price,total_trips_without_hotel_bookings,from_Aracaju (SE),from_Brasilia (DF),from_Campo Grande (MS),from_Florianopolis (SC),from_Natal (RN),from_Recife (PE),from_Rio de Janeiro (RJ),from_Salvador (BH),from_Sao Paulo (SP),to_Aracaju (SE),to_Brasilia (DF),to_Campo Grande (MS),to_Florianopolis (SC),to_Natal (RN),to_Recife (PE),to_Rio de Janeiro (RJ),to_Salvador (BH),to_Sao Paulo (SP),flightType_economic,flightType_firstClass,flightType_premium,agency_CloudFy,agency_FlyingDrops,agency_Rainbow,hotel_Hotel A,hotel_Hotel AF,hotel_Hotel AU,hotel_Hotel BD,hotel_Hotel BP,hotel_Hotel BW,hotel_Hotel CB,hotel_Hotel K,hotel_Hotel Z,place_Aracaju (SE),place_Brasilia (DF),place_Campo Grande (MS),place_Florianopolis (SC),place_Natal (RN),place_Recife (PE),place_Rio de Janeiro (RJ),place_Salvador (BH),place_Sao Paulo (SP),flight_year_2019.0,flight_year_2020.0,flight_year_2021.0,flight_year_2022.0,flight_year_2023.0,flight_month_1.0,flight_month_2.0,flight_month_3.0,flight_month_4.0,flight_month_5.0,flight_month_6.0,flight_month_7.0,flight_month_8.0,flight_month_9.0,flight_month_10.0,flight_month_11.0,flight_month_12.0,flight_is_weekend_False,flight_is_weekend_True,hotel_year_2019.0,hotel_year_2020.0,hotel_year_2021.0,hotel_year_2022.0,hotel_year_2023.0,hotel_month_1.0,hotel_month_2.0,hotel_month_3.0,hotel_month_4.0,hotel_month_5.0,hotel_month_6.0,hotel_month_7.0,hotel_month_8.0,hotel_month_9.0,hotel_month_10.0,hotel_month_11.0,hotel_month_12.0,hotel_is_weekend_False
0,0.0,21,4You,male,89,989.066124,527.233596,1.370337,2.629630,225.741481,62,33,36,8,12,13,35,8,18,15,33,36,8,12,13,35,8,18,15,44,88,46,78,32,68,6,8,4,8,2,4,2,14,6,6,2,4,6,8,4,2,14,8,28,105,45,0,0,19,16,16,18,17,11,9,9,10,18,18,17,136,42,10,36,8,0,0,2,4,0,4,10,4,4,2,4,10,4,6,178
1,1.0,37,4You,male,6,818.898333,508.648333,1.321667,1.000000,193.360000,4,4,2,0,1,0,2,0,1,2,4,2,0,1,0,2,0,1,2,4,2,6,8,2,2,0,2,0,0,2,0,0,0,0,0,2,0,0,0,0,0,0,2,12,0,0,0,0,0,0,0,0,0,0,0,0,2,9,1,0,11,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,12
2,2.0,48,4You,female,131,906.921832,513.671374,1.335191,2.694444,210.050000,95,56,56,12,14,19,50,22,19,14,56,56,12,14,19,50,22,19,14,88,98,76,112,24,126,12,4,6,4,4,14,6,10,12,12,4,14,12,4,6,6,10,4,28,105,104,25,0,27,25,24,18,18,16,19,17,19,28,25,26,197,65,8,30,24,10,0,10,8,6,8,2,0,10,2,2,6,8,10,262
3,3.0,23,4You,female,199,936.913894,506.074372,1.315176,2.300000,217.058500,139,74,87,33,42,23,79,23,17,20,74,87,33,42,23,79,23,17,20,114,172,112,172,56,170,20,12,14,22,4,18,14,12,4,4,4,18,20,22,14,14,12,12,28,105,104,104,57,36,33,34,35,34,35,33,25,28,37,32,36,287,111,8,32,28,24,28,18,14,8,12,10,10,8,4,6,8,12,10,398
4,4.0,44,4You,female,199,957.393417,517.955477,1.345930,2.571429,214.947679,143,73,75,16,28,21,104,32,25,24,73,75,16,28,21,104,32,25,24,96,180,122,164,60,174,14,12,14,10,6,12,22,14,8,8,6,12,14,10,14,22,14,12,28,105,104,104,57,36,32,35,37,33,34,33,25,28,36,33,36,300,98,6,28,26,36,16,12,8,16,12,6,8,6,4,8,8,14,10,398


In [6]:
# dropping the userCode
df = df.drop(columns=['userCode'])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885 entries, 0 to 884
Data columns (total 89 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   age                                 885 non-null    int64  
 1   company                             885 non-null    object 
 2   gender                              885 non-null    object 
 3   total_trips                         885 non-null    int64  
 4   avg_flight_price                    885 non-null    float64
 5   avg_distance                        885 non-null    float64
 6   avg_flight_time                     885 non-null    float64
 7   avg_days                            885 non-null    float64
 8   avg_hotel_price                     885 non-null    float64
 9   total_trips_without_hotel_bookings  885 non-null    int64  
 10  from_Aracaju (SE)                   885 non-null    int64  
 11  from_Brasilia (DF)                  885 non-n

In [8]:
X = df.drop(columns=['gender'])
y = df['gender']

In [15]:
# getting the feature names in X as a list

pre_feature_names = X.columns.tolist()
# saving in artifacts
import pickle 
with open("./artifacts2/pre_feature_names.pkl" , 'wb') as f:
    pickle.dump(pre_feature_names , f)

In [9]:
from sklearn.model_selection import train_test_split

xtrain , xtest ,ytrain , ytest = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [10]:
cat_cols = ['company']
num_cols = [col for col in X.columns if col not in cat_cols]

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


In [12]:
numeric_pipeline = Pipeline(steps = [
    ('power' , PowerTransformer(method = 'yeo-johnson')),
])

categorical_pipeline = Pipeline(steps = [
    ('onehot' , OneHotEncoder(handle_unknown='ignore',sparse_output=False))
])

In [13]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num' , numeric_pipeline , num_cols),
        ('cat' , categorical_pipeline , cat_cols)
    ]
)

In [14]:
xtrain_p = preprocessor.fit_transform(xtrain)
xtest_p = preprocessor.transform(xtest)

print('Train shape after preprocessing : ',xtrain_p.shape)
print('Test shape after preprocessing : ',xtest_p.shape)

Train shape after preprocessing :  (708, 92)
Test shape after preprocessing :  (177, 92)


In [16]:
import pickle 
from pathlib import Path

Path('artifacts2').mkdir(exist_ok=True)

with open('artifacts2/preprocessor.pkl','wb') as f:
    pickle.dump(preprocessor,f)

In [17]:
num_features = preprocessor.named_transformers_["num"].get_feature_names_out(num_cols)
cat_features = preprocessor.named_transformers_["cat"].named_steps["onehot"].get_feature_names_out(cat_cols)

feature_names = list(num_features) + list(cat_features)
feature_names

['age',
 'total_trips',
 'avg_flight_price',
 'avg_distance',
 'avg_flight_time',
 'avg_days',
 'avg_hotel_price',
 'total_trips_without_hotel_bookings',
 'from_Aracaju (SE)',
 'from_Brasilia (DF)',
 'from_Campo Grande (MS)',
 'from_Florianopolis (SC)',
 'from_Natal (RN)',
 'from_Recife (PE)',
 'from_Rio de Janeiro (RJ)',
 'from_Salvador (BH)',
 'from_Sao Paulo (SP)',
 'to_Aracaju (SE)',
 'to_Brasilia (DF)',
 'to_Campo Grande (MS)',
 'to_Florianopolis (SC)',
 'to_Natal (RN)',
 'to_Recife (PE)',
 'to_Rio de Janeiro (RJ)',
 'to_Salvador (BH)',
 'to_Sao Paulo (SP)',
 'flightType_economic',
 'flightType_firstClass',
 'flightType_premium',
 'agency_CloudFy',
 'agency_FlyingDrops',
 'agency_Rainbow',
 'hotel_Hotel A',
 'hotel_Hotel AF',
 'hotel_Hotel AU',
 'hotel_Hotel BD',
 'hotel_Hotel BP',
 'hotel_Hotel BW',
 'hotel_Hotel CB',
 'hotel_Hotel K',
 'hotel_Hotel Z',
 'place_Aracaju (SE)',
 'place_Brasilia (DF)',
 'place_Campo Grande (MS)',
 'place_Florianopolis (SC)',
 'place_Natal (RN)',
 'p

In [18]:
# saving the artifacts

np.save("artifacts2/xtrain_p.npy", xtrain_p)
np.save("artifacts2/xtest_p.npy", xtest_p)
np.save("artifacts2/ytrain.npy", ytrain.values)
np.save("artifacts2/ytest.npy", ytest.values)

# save feature names 

with open('artifacts2/feature_names.pkl','wb') as f:
    pickle.dump(feature_names,f)


print("Artifacts saved successfully!")

Artifacts saved successfully!
